In [1]:
import sys

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [61]:
#Define the columns to be used and pass them to a dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']

Toronto = pd.DataFrame(columns=columns)
Toronto

,PostalCode,Borough,Neighborhood


In [36]:
#Obtain get request for data 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
result = requests.get(url)
print('result obtained')

result obtained


In [37]:
#Use beautiful soupt to scrap the data 
soup = BeautifulSoup(result.content, 'html5lib')
table = soup.find('div', attrs = {'id':'container'})
print('data obtained')

data obtained


In [62]:
#Clean and prepare data

postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [63]:
#lets pass these obtained data to our dataframe

for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neigborhood_name = neighborhoods[data]
    
    Toronto = Toronto.append({'PostalCode':code, 
                             'Borough': borough,
                             'Neighborhood':neigborhood_name}, ignore_index=True)

In [64]:
#Remove \n for each columns
for n in Toronto.columns:
    Toronto[n] = Toronto[n].str.split("\n")
    Toronto[n] = Toronto[n].str[0]

In [65]:
Toronto = Toronto[Toronto['Borough'] != 'Not assigned'].reset_index(drop=True)

In [66]:
Toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


## Question 2

In [73]:
#google geocode kpet running without producing a result

In [67]:
!wget -q -O 'Toronto_location.csv'  http://cocl.us/Geospatial_data
location = pd.read_csv('Toronto_location.csv')
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [68]:
location.columns = ['PostalCode', 'Latitude', 'Longitude']

In [69]:
Toronto_new = Toronto.merge(location, on='PostalCode')

In [72]:
Toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
